In [12]:
import pandas as pd
import numpy as np
import umap
#from bunkatech.semantics.extract_terms import extract_terms_df

In [13]:
from bunkatech.basic_class import BasicSemantics

In [16]:
model = BasicSemantics(data,
                        text_var = 'description',
                        index_var = 'imdb')

In [18]:
model.fit(extract_terms=False,
        terms_embedding=False,
        docs_embedding=True,
        sample_size_terms=500,
        terms_limit=500,
        terms_ents=True,
        terms_ngrams=(1, 2),
        terms_ncs=True,
        terms_include_pos=["NOUN", "PROPN", "ADJ"],
        terms_include_types=["PERSON", "ORG"],
        terms_embedding_model="all-MiniLM-L6-v2",
        docs_embedding_model="all-MiniLM-L6-v2",
        docs_multiprocessing=True,
        terms_multiprocessing = True, 
        docs_dimension_reduction=5,
        language="en")

Start Embedding...
UMAP(n_components=5, verbose=True)
Sun Mar 27 10:43:02 2022 Construct fuzzy simplicial set
Sun Mar 27 10:43:05 2022 Finding Nearest Neighbors
Sun Mar 27 10:43:07 2022 Finished Nearest Neighbor Search
Sun Mar 27 10:43:08 2022 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Sun Mar 27 10:43:11 2022 Finished embedding


In [21]:
docs_embeddings = model.docs_embeddings

In [77]:
import networkx as nx
import multiprocessing
from node2vec import Node2Vec

edge_list = data[['imdb', 'country']]
edge_list['country'] = edge_list['country'].apply(lambda x: x.split(', '))
edge_list = edge_list.explode('country')
edge_list['weight'] = 1
edge_list.columns = ['source', 'target', 'weight']

G = nx.from_pandas_edgelist(
        edge_list, source="source", target="target", edge_attr="weight"
    )

In [78]:
node2vec = Node2Vec(G, dimensions=700, workers=multiprocessing.cpu_count())

Computing transition probabilities:   0%|          | 0/3079 [00:00<?, ?it/s]

Generating walks (CPU: 10): 100%|██████████| 1/1 [00:14<00:00, 14.38s/it]


In [79]:
model = node2vec.fit(window=30, min_count=1)
nodes = list(map(str, G.nodes()))
embeddings = np.array([model.wv[x] for x in nodes])
embeddings = pd.DataFrame(embeddings, index=nodes)
embeddings_red =  umap.UMAP(n_components=5, verbose=True).fit_transform(embeddings)
node_embeddings = pd.DataFrame(embeddings_red, index = nodes)

UMAP(n_components=5, verbose=True)
Sun Mar 27 11:14:16 2022 Construct fuzzy simplicial set
Sun Mar 27 11:14:20 2022 Finding Nearest Neighbors
Sun Mar 27 11:14:20 2022 Finished Nearest Neighbor Search
Sun Mar 27 11:14:20 2022 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Sun Mar 27 11:14:23 2022 Finished embedding


In [80]:
text_emb = docs_embeddings.add_prefix('text_')
node_emb = node_embeddings.add_prefix('node_')

In [81]:
from sklearn.preprocessing import normalize

fin_emb = pd.merge(node_emb, text_emb, left_index=True, right_index=True, how = 'outer')
fin_emb = fin_emb.fillna(fin_emb.mean())
fin_emb = pd.DataFrame(normalize(fin_emb), index = fin_emb.index)

In [84]:
fin_emb_red = umap.UMAP(n_components=2, verbose=True).fit_transform(fin_emb)
fin_emb_red = pd.DataFrame(fin_emb_red, index = fin_emb.index)

UMAP( verbose=True)
Sun Mar 27 11:16:13 2022 Construct fuzzy simplicial set
Sun Mar 27 11:16:16 2022 Finding Nearest Neighbors
Sun Mar 27 11:16:16 2022 Finished Nearest Neighbor Search
Sun Mar 27 11:16:16 2022 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Sun Mar 27 11:16:19 2022 Finished embedding


In [94]:
import plotly.express as px
import bamboolib

fig = px.scatter(fin_emb_red.reset_index(), x=0, y=1, text = 'index', height = 1500, width = 1500)


import plotly
plotly.offline.plot(fig, filename='file.html')

'file.html'